# Various script used mainly in the preprocesssing of the data

In [ ]:
import numpy as np
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path


folder_path = "dataset/topo/embeddings/dnabert"


# list of files in the folder without the extension
IDs = [Path(f).stem for f in listdir(folder_path) if isfile(join(folder_path, f))]


embeddings_matrix = []


for id in IDs:

    # load the file as a numpy array
    raw_embedding = None
    try:
        raw_embedding = np.load(folder_path + "/" + id + ".npy")
        print(raw_embedding.shape)
    except:
        raw_embedding = np.load(folder_path + "/" + id + ".npy", allow_pickle=True)
        print(f"Error while loading the embedding of sequence {id}")
        # os.remove(folder_path + "/" + id + ".npy")
        print(raw_embedding.shape)
        

In [ ]:
import numpy as np
from os import listdir
from os.path import isfile, join
from pathlib import Path
from Bio import SeqIO


FASTA_FILE_PATH = "/storagenfs/m.tolloso/BioEmbedding/dataset/satb2/satb2_tr.txt"
lens = []

for count, seqrecord in enumerate(SeqIO.parse(FASTA_FILE_PATH, "fasta")):

    lens.append(len(str(seqrecord.seq)))

print(count + 1)

print(np.mean(lens))
print(np.max(lens))



In [ ]:
# export the geneIDs given a xml uniprot file
from Bio import SeqIO

records = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/hemoglobin/hemoglobin.xml", "uniprot-xml"))
geneIDs = []

with_gid = 0

xml_names = []
for record in records:
    for i in record.dbxrefs:
        if i.startswith("GeneID:"):
            geneID = i.split(":")[1]
            geneIDs.append(geneID)
            with_gid += 1
            break
    


print(with_gid)

# with open("emoglobina_ids.txt", 'w') as f:
#     for i in geneIDs:
#         f.write(i + "\n")



In [ ]:
# given a genbank(full) file, for each proteome select the superset proteins (CDS) and add them to a fasta file

from Bio import SeqIO
import Bio

no_ids = 0

def to_simple_location(f_list):
    for feature in f_list: 
        if type(feature.location) == Bio.SeqFeature.CompoundLocation:
            feature.location = (int(feature.location.parts[0].start) , int(feature.location.parts[1].end))
        else:
            feature.location = (int(feature.location.start), int(feature.location.end))

def remove_subsets(f_list):
    for i in range(len(f_list)):
        for j in range(i+1, len(f_list)):
            if f_list[i].location[0] <= f_list[j].location[0] and f_list[i].location[1] >= f_list[j].location[1]:
                # print(f'removed protein {f_list[j].qualifiers["protein_id"]}')
                f_list.pop(j)
                break
            elif f_list[j].location[0] <= f_list[i].location[0] and f_list[j].location[1] >= f_list[i].location[1]:
                # print(f'removed protein {f_list[i].qualifiers["protein_id"]}')
                f_list.pop(i)
                break
            else:
                continue
        break
    return f_list


gb_records = list(SeqIO.parse('dataset/covid19/covid19.gb','genbank'))
final_proteins = []

for gb_record in gb_records:
    CDS_features = []

    for feature in gb_record.features:
        if feature.type == 'CDS':
            CDS_features.append(feature) 

    to_simple_location(CDS_features)

    # print(len(CDS_features))
    remove_subsets(CDS_features)
    # print(len(CDS_features))

    for feature in CDS_features:
        # print(feature.location, feature.qualifiers['protein_id'], feature.qualifiers['translation'])

        try:
            final_proteins.append((feature.qualifiers['protein_id'], feature.qualifiers['translation']))
        except:
            no_ids +=1


print(len(final_proteins))
print(no_ids)  
 

# output_file = f"dataset/covid19/covid19_proteins.fasta"

# with open(output_file, 'w') as fasta_file:
#     for entry in final_proteins:
#         description, sequence = entry
#         fasta_file.write(f'>{description[0]}\n{sequence[0]}\n')




In [ ]:
from Bio import SeqIO
import numpy as np

gb_records = list(SeqIO.parse('/storagenfs/m.tolloso/BioEmbedding/dataset/meningite/meningite_tr.fasta','fasta'))

lens = []

for gb_record in gb_records:
    lens.append(len(gb_record.seq))

print(np.min(lens))

In [ ]:
from Bio import SeqIO
import os
import numpy as np 

path = '/storagenfs/m.tolloso/BioEmbedding/dataset/covid19/embeddings/alphafold'

# for each file in the folder 'path'

ids = os.listdir(path)


tensor1 = np.load(path + '/' + ids[0])
tensor2 = np.load(path + '/' + ids[3])

print(tensor1.shape)
print(tensor2.shape)

# concat the tensors over the second axis

concat = np.concatenate((tensor1, tensor2), axis=1)

print(concat.shape)

In [ ]:
from Bio import SeqIO
import os
import numpy as np 

records = list(SeqIO.parse('dataset/batterio/batterio_tr.fasta','fasta'))

geneIDS = []

for record in records:
    id = record.id
    geneID = record.description.split("[GeneID=")[1].split(']')[0]
    # geneIDS.append(geneID)

    os.rename(f'dataset/batterio/embeddings/dnabert/{id}.npy', f'dataset/batterio/embeddings/dnabert/{geneID}.npy')
    # print(id, geneID)

# print(len(geneIDS))
# print(len(set(geneIDS)))




In [ ]:
import re

def extract_gene_id(fasta_header):
    match = re.search(r'\[GeneID=(\d+)\]', fasta_header)
    return match.group(1) if match else None


def process_fasta_file(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
        for line in input_file:
            if line.startswith('>'):

                gene_id = extract_gene_id(line)
                
                # Modify the header to include GeneID
                modified_header = line.replace('>', f'>{gene_id} ')
                output_file.write(modified_header)
                
                # Update current header for the new sequence
                current_header = line.strip()
            else:
                # Continue writing sequence lines
                output_file.write(line)



# Replace 'input.fasta' and 'output.fasta' with the actual file paths
input_file_path = '/storagenfs/m.tolloso/BioEmbedding/dataset/batterio/batterio_tr.fasta'
output_file_path = '/storagenfs/m.tolloso/BioEmbedding/dataset/batterio/batterio_tr_new.fasta'
process_fasta_file(input_file_path, output_file_path)


In [ ]:
from Bio import SeqIO
import os

records = list(SeqIO.parse('dataset/mouse/mouse_tr.fasta','fasta'))
files = os.listdir('dataset/mouse/embeddings/dnabert')

ids = [record.id for record in records]
embs = [file.split('.')[0] for file in files]


for id in ids:
    if id not in embs:
        print(id)  





In [ ]:
# aggegate the satb fasta files
from Bio import SeqIO
from pathlib import Path


files = ['Satb1_Orthologues_protein_sequences.txt', 'Satb1_Satb2_protein_sequences.txt', 'Satb2_Orthologues_protein_sequences.txt']
files_tr = ['Satb1_Orthologues_tr_sequences.txt', 'Satb1_Satb2_tr_sequences.txt', 'Satb2_Orthologues_tr_sequences.txt']

transcript_dict = {}

for file_tr in files_tr:
    records_fasta = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/bacterium_proteins.faa", "fasta"))

    for record_tr in records_fasta:
        transcript_dict[record_tr.id] = record_tr.description.split('[GeneID=')[1][:-1]



embedder = 'alphafold'



files = os.listdir(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}')
for file in files :

    # print(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{protein_name}.npy')
    # print(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{transcript_dict[protein_name]}.npy')


    os.rename(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{file}', 
                f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{file}'[:-8])

          

transcript_dict

# with open('dataset/satb2/satb2_list.txt', 'w') as of:

#     for file in files:
#         records = list(SeqIO.parse(f'dataset/satb2/{file}', 'fasta'))
        
#         for record in records:
            
#             if len(record.seq) == 0:
#                 of.write(f'>gene:{record.id}')
#                 seq = transcript_dict[record.id]
#             else:
#                 of.write(f' protein:{record.id}\n')
#                 of.write(f'{seq}\n')
                 

In [ ]:
from Bio import SeqIO

file_name = f'dataset/satb2/satb2_tr.txt'

import FastaValidator

return_code = FastaValidator.fasta_validator(file_name)
print(return_code)



records_tr = list(SeqIO.parse(f'dataset/satb2/satb2_tr.txt', 'fasta'))


ids = []

for record in records_tr:
    print(record.id)



In [ ]:
# export the geneIDs given a xml uniprot file
from Bio import SeqIO
from autoembedding.utils import get_gene_id
import os

# records_xml = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/Neisseria_meningitidis.xml", "uniprot-xml"))
records_fasta = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/bacterium_tr_new.fasta", "fasta"))



complete = 0

# for record in records_fasta:
#     fasta_IDs.append(record.id)

for record in records_fasta:


    os.path.exists(f"/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/alphafold/{record.id}.npy")
    complete += 1


print(complete)


In [ ]:
final = list(set(xml_IDs).intersection(set(fasta_IDs)))
len(final)

In [ ]:
from Bio import SeqIO
with open('dataset/bacterium/bacterium_tr_new.fasta', 'w') as f:

    for record in records_fasta:
        if record.id in final:
            # write the record to the output file
            SeqIO.write(record, f, "fasta")

In [ ]:
from Bio import SeqIO
from itertools import combinations


command = './clustalo-1.2.4-Ubuntu-x86_64 --infile tmp/test.fasta --threads 40 --MAC-RAM 8000 -v --verbose --guidetree-out tmp/gt.dnd --distmat-out tmp/dm.matrix --outfmt clustal --resno --outfile tmp/of.clustal_num --output-order tree-order --seqtype dna'
command_nodm = './clustalo-1.2.4-Ubuntu-x86_64 --infile tmp/test.fasta --threads 40 -v --verbose --auto --outfile tmp/of.clustal_num  --seqtype dna'


mengite_tr = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/meningite/meningite_tr.fasta", "fasta"))



for record1, record2 in combinations(mengite_tr, 2):
    with open("test.fasta", "w") as f:
        SeqIO.write([record1, record2], f, "fasta")
        break


In [ ]:

import pickle
from scipy.cluster import hierarchy

res = pickle.load(open("tmp/res_covid19.pkl", "rb"))
Z = res['embeddings_linkage_matrix']
leaf_names = res['embeddings_IDs']


def get_newick(node, parent_dist, leaf_names, newick='') -> str:
    """
    Convert sciply.cluster.hierarchy.to_tree()-output to Newick format.

    :param node: output of sciply.cluster.hierarchy.to_tree()
    :param parent_dist: output of sciply.cluster.hierarchy.to_tree().dist
    :param leaf_names: list of leaf names
    :param newick: leave empty, this variable is used in recursion.
    :returns: tree in Newick format
    """
    if node.is_leaf():
        return "%s:%.2f%s" % (leaf_names[node.id], parent_dist - node.dist, newick)
    else:
        if len(newick) > 0:
            newick = "):%.2f%s" % (parent_dist - node.dist, newick)
        else:
            newick = ");"
        newick = get_newick(node.get_left(), node.dist, leaf_names, newick=newick)
        newick = get_newick(node.get_right(), node.dist, leaf_names, newick=",%s" % (newick))
        newick = "(%s" % (newick)
        return newick

tree = hierarchy.to_tree(Z, False)
newick = get_newick(tree, tree.dist, leaf_names)
with open("results_trees/covid19_esm.nwk", "w") as f:
    f.write(newick)

In [ ]:
# check the proteins of the meningits (for length variance etc.)
from Bio import SeqIO

gb_records = list(SeqIO.parse('dataset/meningite/meningite.gb','genbank'))
proteins = {}

for gb_record in gb_records[:2]:

    for feature in gb_record.features:
        if feature.type == 'CDS':
            try:
                protein_sequence = feature.qualifiers['translation'][0]
                protein_id = feature.qualifiers['product'][0]
            except:
                continue
            try:
                proteins[protein_id] += 1
            except:
                proteins[protein_id] = 1


    

print(len(proteins))





In [ ]:
for protein in proteins.keys():

    print(protein, (proteins[protein]))

In [ ]:
# read the phylogenetic distance matrix, build a UPGMA phylogenetic tree, and export it as a newik tree
 
from autoembedding import utils
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage
from scipy.cluster import hierarchy

case_study = "satb2"

method = "average"
metric = "euclidean"

gtrue_IDs, gt_distances = utils.read_distance_matrix(case_study)
gt_distances = squareform(gt_distances)
gtrue_linkage_matrix = linkage(gt_distances, method=method, metric=metric)

tree = hierarchy.to_tree(gtrue_linkage_matrix, False)

newick = utils.get_newick(tree, tree.dist, gtrue_IDs)

with open(f"results_trees/{case_study}_upgma.nwk", "w") as f:
    f.write(newick)
